# Step 0.1. Import necessary libraries 

In [1]:
# Standard python libraries
import logging
import os
import time
logging.basicConfig(format='[%(asctime)s] (%(levelname)s): %(message)s', level=logging.INFO)

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import torch

# Imports from our package
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.dataset.roles import DatetimeRole
from lightautoml.tasks import Task
from lightautoml.utils.profiler import Profiler

# Step 0.2. Parameters 

In [2]:
N_THREADS = 8 # threads cnt for lgbm and linear models
N_FOLDS = 5 # folds cnt for AutoML
RANDOM_STATE = 42 # fixed random state for various reasons
TEST_SIZE = 0.2 # Test size for metric check
TIMEOUT = 300 # Time in seconds for automl run
TARGET_NAME = 'TARGET' # Target column name

# Step 0.3. Fix torch number of threads and numpy seed 

In [3]:
np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

# Step 0.4. Change profiling decorators settings 

By default, profiling decorators are turned off for speed and memory reduction. If you want to see profiling report after using LAMA, you need to turn on the decorators using command below: 

In [4]:
p = Profiler()
p.change_deco_settings({'enabled': True})

ALL_FUNCS len = 418


# Step 0.5. Example data load 

In [5]:
%%time

data = pd.read_csv('./example_data/test_data_files/sampled_app_train.csv')
data.head()

CPU times: user 80.8 ms, sys: 4.53 ms, total: 85.4 ms
Wall time: 84.7 ms


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,313802,0,Cash loans,M,N,Y,0,270000.0,327024.0,15372.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,319656,0,Cash loans,F,N,N,0,108000.0,675000.0,19737.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,207678,0,Revolving loans,F,Y,Y,2,112500.0,270000.0,13500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
3,381593,0,Cash loans,F,N,N,1,67500.0,142200.0,9630.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
4,258153,0,Cash loans,F,Y,Y,0,337500.0,1483231.5,46570.5,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,0.0


# Step 0.6. (Optional) Some user feature preparation 

Cell below shows some user feature preparations to create task more difficult (this block can be omitted if you don't want to change the initial data):

In [6]:
%%time

data['BIRTH_DATE'] = (np.datetime64('2018-01-01') + data['DAYS_BIRTH'].astype(np.dtype('timedelta64[D]'))).astype(str)
data['EMP_DATE'] = (np.datetime64('2018-01-01') + np.clip(data['DAYS_EMPLOYED'], None, 0).astype(np.dtype('timedelta64[D]'))
                    ).astype(str)

data['constant'] = 1
data['allnan'] = np.nan

data['report_dt'] = np.datetime64('2018-01-01')

data.drop(['DAYS_BIRTH', 'DAYS_EMPLOYED'], axis=1, inplace=True)

[2020-11-11 22:33:41,141] (INFO): Note: detected 96 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2020-11-11 22:33:41,142] (INFO): Note: NumExpr detected 96 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[2020-11-11 22:33:41,142] (INFO): NumExpr defaulting to 8 threads.


CPU times: user 112 ms, sys: 3.61 ms, total: 116 ms
Wall time: 114 ms


# Step 0.7. (Optional) Data splitting for train-test 

Block below can be omitted if you are going to train model only or you have specific train and test files:

In [7]:
%%time

train_data, test_data = train_test_split(data, 
                                         test_size=TEST_SIZE, 
                                         stratify=data[TARGET_NAME], 
                                         random_state=RANDOM_STATE)
logging.info('Data splitted. Parts sizes: train_data = {}, test_data = {}'
              .format(train_data.shape, test_data.shape))

[2020-11-11 22:33:41,220] (INFO): Data splitted. Parts sizes: train_data = (8000, 125), test_data = (2000, 125)


CPU times: user 15.4 ms, sys: 0 ns, total: 15.4 ms
Wall time: 14.1 ms


In [8]:
train_data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,BIRTH_DATE,EMP_DATE,constant,allnan,report_dt
6444,112261,0,Cash loans,F,N,N,1,90000.0,640080.0,31261.5,...,0.0,0.0,0.0,1.0,0.0,1985-06-28,2012-06-21,1,NaN,2018-01-01
3586,115058,0,Cash loans,F,N,Y,0,180000.0,239850.0,23850.0,...,0.0,0.0,0.0,0.0,3.0,1953-12-27,2018-01-01,1,NaN,2018-01-01
9349,326623,0,Cash loans,F,N,Y,0,112500.0,337500.0,31086.0,...,0.0,0.0,0.0,0.0,2.0,1975-06-21,2016-06-17,1,NaN,2018-01-01
7734,191976,0,Cash loans,M,Y,Y,1,67500.0,135000.0,9018.0,...,NaN,NaN,NaN,NaN,NaN,1988-04-27,2009-06-05,1,NaN,2018-01-01
2174,281519,0,Revolving loans,F,N,Y,0,67500.0,202500.0,10125.0,...,0.0,0.0,0.0,0.0,2.0,1975-06-13,1997-01-22,1,NaN,2018-01-01


# ========= AutoML preset usage =========


## Step 1. Create Task

In [9]:
%%time

task = Task('binary', )

CPU times: user 1.66 ms, sys: 2.46 ms, total: 4.13 ms
Wall time: 3.74 ms


## Step 2. Setup columns roles

Roles setup here set target column and base date, which is used to calculate date differences:

In [10]:
%%time

roles = {'target': TARGET_NAME,
         DatetimeRole(base_date=True, seasonality=(), base_feats=False): 'report_dt',
         }

CPU times: user 3 µs, sys: 191 µs, total: 194 µs
Wall time: 197 µs


## Step 3. Create AutoML from preset

To create AutoML model here we use `TabularAutoML` preset, which looks like:

![TabularAutoML preset pipeline](imgs/tutorial_2_pipeline.png)

All params we set above can be send inside preset to change its configuration:

In [11]:
%%time 

automl = TabularAutoML(task = task, 
                       timeout = TIMEOUT,
                       general_params = {'nested_cv': False, 'use_algos': [['linear_l2', 'lgb', 'lgb_tuned']]},
                       reader_params = {'cv': N_FOLDS, 'random_state': RANDOM_STATE},
                       tuning_params = {'max_tuning_iter': 20, 'max_tuning_time': 30},
                       lgb_params = {'default_params': {'n_threads': N_THREADS}})
oof_pred = automl.fit_predict(train_data, roles = roles)
logging.info('oof_pred:\n{}\nShape = {}'.format(oof_pred, oof_pred.shape))

/mnt/alex/LightAutoML/lightautoml/utils/timer.py:210: UserWarning:

Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer

/mnt/alex/LightAutoML/lightautoml/reader/guess_roles.py:351: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Feats was rejected during automatic roles guess: []
Train process start. Time left 295.983624458313 secs
C = 1e-05 score = 0.6871107761224348
C = 5e-05 score = 0.7049836693020297
C = 0.0001 score = 0.7203521657552481
C = 0.0005 score = 0.7541307845684259
C = 0.001 score = 0.7616252652736126
C = 0.005 score = 0.7634160292081617
C = 0.01 score = 0.7601766174340223
C = 0.05 score = 0.7482025541104714
C = 1e-05 score = 0.7097380264945652
C = 5e-05 score = 0.7196469514266305
C = 0.0001 score = 0.726806640625
C = 0.0005 score = 0.7401706861413043
C = 0.001 score = 0.7423785665760868
C = 0.005 score = 0.7421981148097827
C = 0.01 score = 0.7413011633831522
C = 1e-05 score = 0.6369283924932064
C = 5e-05 score = 0.6511761209239131
C = 0.0001 score = 0.6631337041440217
C = 0.0005 score = 0.6919688349184784
C = 0.001 score = 0.7008268936820652
C = 0.005 score = 0.7106243631114132
C = 0.01 score = 0.7092550526494564
C = 0.05 score = 0.7000944718070651
C = 1e-05 score = 0.6700306768002717
C = 5e-05 

/mnt/alex/LightAutoML/lightautoml/utils/timer.py:210: UserWarning:

Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer

[I 2020-11-11 22:34:06,599] A new study created in memory with name: no-name-0ef90daa-9fe7-4ddb-8544-8f6d4196180c


Early stopping, best iteration is:
[366]	valid's auc: 0.759569
Time history [2.13704252243042, 2.0840117931365967, 3.077803134918213, 2.9917008876800537, 3.772228956222534, 2.5856049060821533]. Time left 50.28570532798767
Optuna may run 225.87924909591675 secs
Training until validation scores don't improve for 200 rounds
[100]	valid's auc: 0.76417
[200]	valid's auc: 0.76039
[300]	valid's auc: 0.759968


[I 2020-11-11 22:34:09,867] Trial 0 finished with value: 0.7660460466881558 and parameters: {'feature_fraction': 0.6872700594236812, 'num_leaves': 108}. Best is trial 0 with value: 0.7660460466881558.


...

Training until validation scores don't improve for 200 rounds
[100]	valid's auc: 0.740035
[200]	valid's auc: 0.742253
[300]	valid's auc: 0.747395
[400]	valid's auc: 0.749993
[500]	valid's auc: 0.74886


[I 2020-11-11 22:34:36,684] Trial 7 finished with value: 0.7507630792586771 and parameters: {'feature_fraction': 0.5282057895135501, 'num_leaves': 103}. Best is trial 5 with value: 0.766142266839863.


Early stopping, best iteration is:
[382]	valid's auc: 0.750763
Time history [5.148114204406738]. Time left 8999999994.84999
Training until validation scores don't improve for 100 rounds
[100]	valid's auc: 0.737469
Early stopping, best iteration is:
[23]	valid's auc: 0.741045
Training until validation scores don't improve for 100 rounds
[100]	valid's auc: 0.725087
[200]	valid's auc: 0.735203
Early stopping, best iteration is:
[182]	valid's auc: 0.737161
Training until validation scores don't improve for 100 rounds
[100]	valid's auc: 0.718368
[200]	valid's auc: 0.719291
[300]	valid's auc: 0.719408
Early stopping, best iteration is:
[212]	valid's auc: 0.721218
Training until validation scores don't improve for 100 rounds
[100]	valid's auc: 0.734911
[200]	valid's auc: 0.739258
Early stopping, best iteration is:
[194]	valid's auc: 0.739842
Training until validation scores don't improve for 100 rounds
[100]	valid's auc: 0.746269
[200]	valid's auc: 0.748355
Early stopping, best iteration is:


[2020-11-11 22:34:47,094] (INFO): oof_pred:
array([[0.02656044],
       [0.02681707],
       [0.03005865],
       ...,
       [0.02124858],
       [0.164036  ],
       [0.09136057]], dtype=float32)
Shape = (8000, 1)


Blending, iter 2: score = 0.7513083907298949, weights = [0.34694588 0.6530542  0.        ]
No score update. Terminated
CPU times: user 7min 49s, sys: 3.89 s, total: 7min 53s
Wall time: 1min 5s


## Step 4. Predict to test data and check scores

In [12]:
%%time

test_pred = automl.predict(test_data)
logging.info('Prediction for test data:\n{}\nShape = {}'
              .format(test_pred, test_pred.shape))

logging.info('Check scores...')
logging.info('OOF score: {}'.format(roc_auc_score(train_data[TARGET_NAME].values, oof_pred.data[:, 0])))
logging.info('TEST score: {}'.format(roc_auc_score(test_data[TARGET_NAME].values, test_pred.data[:, 0])))

[2020-11-11 22:34:47,535] (INFO): Prediction for test data:
array([[0.06316119],
       [0.07426225],
       [0.02580068],
       ...,
       [0.05014627],
       [0.04235209],
       [0.2220479 ]], dtype=float32)
Shape = (2000, 1)
[2020-11-11 22:34:47,536] (INFO): Check scores...
[2020-11-11 22:34:47,540] (INFO): OOF score: 0.7513083907298949
[2020-11-11 22:34:47,542] (INFO): TEST score: 0.731970108695652


CPU times: user 1.12 s, sys: 7.59 ms, total: 1.12 s
Wall time: 443 ms


## Step 5. Profiling AutoML 

To build report here, we **must** turn on decorators on step 0.4. Report is interactive and you can go as deep into functions call stack as you want:

In [13]:
%%time
p.profile('my_report_profile.html')
assert os.path.exists('my_report_profile.html'), 'Profile report failed to build'

FULL_STATS_DF shape = (8325, 6)
RUN_FNAME vc head:
PandasToPandasReader._is_ok_feature [111]    1
LAMLDataset._get_cols_idx [26]               1
NumpyDataset._get_2d [24]                    1
CatLinear.forward [349]                      1
CatLogisticRegression.forward [568]          1
Name: run_fname, dtype: int64
CONNECTED COMPONENTS cnt = 1
PATH LENS describe:
count    8326.000000
mean        9.012251
std         2.779780
min         0.000000
25%         7.000000
50%         9.000000
75%        10.000000
max        21.000000
dtype: float64
CPU times: user 1.02 s, sys: 28.2 ms, total: 1.04 s
Wall time: 1.04 s


## Step 6. Create AutoML with time utilization 

Below we are going to create specific AutoML preset for TIMEOUT utilization (try to spend it as much as possible):

In [14]:
%%time 

automl = TabularUtilizedAutoML(task = task, 
                       timeout = TIMEOUT,
                       general_params = {'nested_cv': False, 'use_algos': [['linear_l2', 'lgb', 'lgb_tuned']]},
                       reader_params = {'cv': N_FOLDS, 'random_state': RANDOM_STATE},
                       tuning_params = {'max_tuning_iter': 20, 'max_tuning_time': 30},
                       lgb_params = {'default_params': {'n_threads': N_THREADS}})
oof_pred = automl.fit_predict(train_data, roles = roles)
logging.info('oof_pred:\n{}\nShape = {}'.format(oof_pred, oof_pred.shape))

CUR SETUP FOR RANDOM STATE: {'reader_params': {'random_state': 42}}
FOUND reader_params in kwargs, need to combine
MERGED VARIANT FOR reader_params = {'cv': 5, 'random_state': 42}


/mnt/alex/LightAutoML/lightautoml/reader/guess_roles.py:351: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Feats was rejected during automatic roles guess: []
Train process start. Time left 298.7156345844269 secs
C = 1e-05 score = 0.6871107761224348
C = 5e-05 score = 0.7049836693020297
C = 0.0001 score = 0.7231478957187378
C = 0.0005 score = 0.7541949313362306
C = 0.001 score = 0.7611922745909308
C = 0.005 score = 0.763330500184422
C = 0.01 score = 0.7603904399933715
C = 0.05 score = 0.7483094653901461
C = 1e-05 score = 0.7097380264945652
C = 5e-05 score = 0.7196469514266305
C = 0.0001 score = 0.726806640625
C = 0.0005 score = 0.7401706861413043
C = 0.001 score = 0.7423785665760868
C = 0.005 score = 0.7421025815217392
C = 0.01 score = 0.7413117781929349
C = 1e-05 score = 0.6368514351222825
C = 5e-05 score = 0.6511761209239131
C = 0.0001 score = 0.6631337041440217
C = 0.0005 score = 0.6919582201086957
C = 0.001 score = 0.7008322010869565
C = 0.005 score = 0.7106509001358695
C = 0.01 score = 0.7091542119565217
C = 0.05 score = 0.7000520125679347
C = 1e-05 score = 0.6700306768002717
C = 5e-05 

/mnt/alex/LightAutoML/lightautoml/utils/timer.py:210: UserWarning:

Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer

[I 2020-11-11 22:35:10,678] A new study created in memory with name: no-name-e3cd1756-317e-4f58-b7b3-8774a86024fd


[600]	valid's auc: 0.755339
Early stopping, best iteration is:
[431]	valid's auc: 0.757653
Time history [2.1371512413024902, 3.0651638507843018, 3.2649903297424316, 5.082632064819336, 3.0418026447296143]. Time left 49.527692681550974
Optuna may run 222.53812839984894 secs
Training until validation scores don't improve for 200 rounds
[100]	valid's auc: 0.754788
[200]	valid's auc: 0.755815
[300]	valid's auc: 0.758348
[400]	valid's auc: 0.759685
[500]	valid's auc: 0.757055


...

[100]	valid's auc: 0.717984
[200]	valid's auc: 0.720545
[300]	valid's auc: 0.723025
[400]	valid's auc: 0.729931
[500]	valid's auc: 0.728793
[600]	valid's auc: 0.729686
Early stopping, best iteration is:
[413]	valid's auc: 0.730445


[I 2020-11-11 22:38:55,091] Trial 5 finished with value: 0.7304419177745349 and parameters: {'feature_fraction': 0.6668543055695109, 'num_leaves': 119}. Best is trial 1 with value: 0.7427714611030037.


Time history [6.144635915756226]. Time left 8999999993.853521
Training until validation scores don't improve for 100 rounds
[100]	valid's auc: 0.731407
Early stopping, best iteration is:
[22]	valid's auc: 0.736378
Training until validation scores don't improve for 100 rounds
[100]	valid's auc: 0.761989
[200]	valid's auc: 0.754034
Early stopping, best iteration is:
[115]	valid's auc: 0.764426
Training until validation scores don't improve for 100 rounds
[100]	valid's auc: 0.731663
[200]	valid's auc: 0.736158
[300]	valid's auc: 0.738663
[400]	valid's auc: 0.73722
Early stopping, best iteration is:
[303]	valid's auc: 0.739693
Training until validation scores don't improve for 100 rounds
[100]	valid's auc: 0.725474
[200]	valid's auc: 0.723511
Early stopping, best iteration is:
[106]	valid's auc: 0.727316
Training until validation scores don't improve for 100 rounds
[100]	valid's auc: 0.706177
[200]	valid's auc: 0.707228
Early stopping, best iteration is:
[141]	valid's auc: 0.711288
Time hi

[2020-11-11 22:39:05,484] (INFO): oof_pred:
array([[0.03720199],
       [0.03239563],
       [0.03478435],
       ...,
       [0.03037913],
       [0.15554607],
       [0.09786935]], dtype=float32)
Shape = (8000, 1)


Blending, iter 4: score = 0.7575391518001122, weights = [0.31697652 0.12107207 0.56195146 0.        ]
No score update. Terminated
CPU times: user 32min 10s, sys: 11.4 s, total: 32min 21s
Wall time: 4min 16s


## Step 7. Predict to test data and check scores for utilized automl

In [15]:
%%time

test_pred = automl.predict(test_data)
logging.info('Prediction for test data:\n{}\nShape = {}'
              .format(test_pred, test_pred.shape))

logging.info('Check scores...')
logging.info('OOF score: {}'.format(roc_auc_score(train_data[TARGET_NAME].values, oof_pred.data[:, 0])))
logging.info('TEST score: {}'.format(roc_auc_score(test_data[TARGET_NAME].values, test_pred.data[:, 0])))

[2020-11-11 22:39:07,077] (INFO): Prediction for test data:
array([[0.0595268 ],
       [0.08256021],
       [0.03199889],
       ...,
       [0.04806679],
       [0.04075336],
       [0.20718434]], dtype=float32)
Shape = (2000, 1)
[2020-11-11 22:39:07,078] (INFO): Check scores...
[2020-11-11 22:39:07,081] (INFO): OOF score: 0.757617388431481
[2020-11-11 22:39:07,083] (INFO): TEST score: 0.7355808423913044


CPU times: user 3.98 s, sys: 240 ms, total: 4.22 s
Wall time: 1.59 s


## Step 8. Profiling utilized AutoML 

To build report here, we **must** turn on decorators on step 0.4. Report is interactive and you can go as deep into functions call stack as you want:

In [16]:
%%time
p.profile('my_report_profile.html')
assert os.path.exists('my_report_profile.html'), 'Profile report failed to build'

FULL_STATS_DF shape = (34506, 6)
RUN_FNAME vc head:
Loss._fw_func [53]                      1
LAMLDataset._initialize [738]           1
NumpyDataset._check_dtype [904]         1
CatLogisticRegression.forward [1368]    1
LAMLDataset.set_data [245]              1
Name: run_fname, dtype: int64
CONNECTED COMPONENTS cnt = 1
PATH LENS describe:
count    34507.000000
mean         9.721680
std          2.785337
min          0.000000
25%          8.000000
50%         10.000000
75%         11.000000
max         22.000000
dtype: float64
CPU times: user 1.66 s, sys: 100 ms, total: 1.76 s
Wall time: 1.75 s


# Appendix. Profiling report screenshots 

After loading HTML with profiling report, you can see fully folded report (please wait for green LOAD OK text for full load finish). If you click on triangle on the left, it unfolds and look like this:  

<img src="imgs/tutorial_2_initial_report.png" alt="Initial profiling report" style="width: 500px;"/>

If we go even deeper we will receive situation like this:

<img src="imgs/tutorial_2_unfolded_report.png" alt="Profiling report after several unfoldings on different levels" style="width: 600px;"/>
